In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
dta = sm.datasets.fair.load_pandas().data

In [4]:
dta.head(5)

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0.111111
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3.230769
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1.400000
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0.727273
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4.666666


In [5]:
# add "affair" column: 1 represents having affairs, 0 represents not
dta['affair'] = (dta.affairs > 0).astype(int)
y, X = dmatrices('affair ~ rate_marriage + age + yrs_married + children + religious + educ + C(occupation) + C(occupation_husb)',
dta, return_type="dataframe")
X.columns

Index(['Intercept', 'C(occupation)[T.2.0]', 'C(occupation)[T.3.0]',
       'C(occupation)[T.4.0]', 'C(occupation)[T.5.0]', 'C(occupation)[T.6.0]',
       'C(occupation_husb)[T.2.0]', 'C(occupation_husb)[T.3.0]',
       'C(occupation_husb)[T.4.0]', 'C(occupation_husb)[T.5.0]',
       'C(occupation_husb)[T.6.0]', 'rate_marriage', 'age', 'yrs_married',
       'children', 'religious', 'educ'],
      dtype='object')

In [6]:
X = X.rename(columns = {'C(occupation)[T.2.0]':'occ_2',

'C(occupation)[T.3.0]':'occ_3',
'C(occupation)[T.4.0]':'occ_4',
'C(occupation)[T.5.0]':'occ_5',
'C(occupation)[T.6.0]':'occ_6',
'C(occupation_husb)[T.2.0]':'occ_husb_2',
'C(occupation_husb)[T.3.0]':'occ_husb_3',
'C(occupation_husb)[T.4.0]':'occ_husb_4',
'C(occupation_husb)[T.5.0]':'occ_husb_5',
'C(occupation_husb)[T.6.0]':'occ_husb_6'})

y = np.ravel(y)

In [7]:
y

array([ 1.,  1.,  1., ...,  0.,  0.,  0.])

In [8]:
y = pd.DataFrame(y)

In [9]:
dta.affair.value_counts()

0    4313
1    2053
Name: affair, dtype: int64

In [10]:
X_Train,X_Test,Y_Train,Y_Test = train_test_split(X,y,test_size = 0.3,random_state = 4)

In [11]:
m1 = LogisticRegression()
m1.fit(X_Train,Y_Train)
from sklearn.metrics import accuracy_score,recall_score,precision_score


E:\anaconda\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
print(accuracy_score(Y_Test,m1.predict(X_Test)))
print(m1.score(X_Train,Y_Train))
print(recall_score(Y_Test,m1.predict(X_Test)))

0.712041884817
0.731149012567
0.331699346405


In [19]:
predicted = m1.predict(X_Test)
predicted

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [20]:
pd.value_counts(predicted)

0.0    1566
1.0     344
dtype: int64

In [22]:
print(metrics.confusion_matrix(Y_Test, predicted))
print(metrics.classification_report(Y_Test, predicted))

[[1157  141]
 [ 409  203]]
             precision    recall  f1-score   support

        0.0       0.74      0.89      0.81      1298
        1.0       0.59      0.33      0.42       612

avg / total       0.69      0.71      0.69      1910

